Pre-req:

In [1]:
import findspark
findspark.init()

import datetime
import pickle
from urllib.request import urlopen
import os
import pandas as pd
import numpy as np
import gensim
import pickle
import matplotlib.pyplot as plt
import pyspark

import heapq

In [2]:
df = pd.read_csv('https://raw.githubusercontent.com/Galeforse/DST-Assessment-05/main/Data/NCSC%20Reports.csv')

In [3]:
print("shape:"+str(np.shape(df)))
df.head()

shape:(221, 5)


,Unnamed: 0,Title,Article,topics,Links
0,0,23rd April 2021,['The NCSC is aware that a malicious piece of ...,"['Cyber attack', 'Cyber strategy', 'Education'...",https://www.ncsc.gov.uk/report/weekly-threat-r...
1,1,16th April 2021,['Cyber security researchers have uncovered a ...,"['Cyber strategy', 'Patching', 'Vulnerabilities']",https://www.ncsc.gov.uk/report/weekly-threat-r...
2,2,12th April 2021,"['Cyber security researchers, Esentire, have w...","['Phishing', 'Social media', 'Personal data', ...",https://www.ncsc.gov.uk/report/weekly-threat-r...
3,3,2nd April 2021,['The UK education sector continues to face an...,"['Education', 'Incident management', 'Secure d...",https://www.ncsc.gov.uk/report/weekly-threat-r...
4,4,26th March 2021,['Earlier this month Microsoft confirmed that ...,"['Cyber attack', 'Education', 'Mitigation', 'P...",https://www.ncsc.gov.uk/report/weekly-threat-r...


In [4]:
df.iloc[67,2]

"['The government has issued a cyber security alert to charities warning them of a spike in the number of criminals trying to access and change the private information of staff.\\nThe Charity Commission has received several reports from charities that have been targeted by fraudsters impersonating HR staff members, specifically attempting to change employees bank details. In all cases, the request was made through an email.\\nCharities should look out for email requests from spoofed or similar email addresses to their legitimate HR departments, finance departments or staff with authority to change bank details. The NCSC has provided guidance which can help with this.\\nLike most businesses, charities are increasingly reliant on computer technology and are at risk of falling victim to cyber criminals. The NCSC has published guidance on how to improve cyber security within a charity with the collection including specific phishing advice.', 'With Christmas shopping well underway, this wee

In [5]:
df.iloc[67,3]

"['Cyber threat']"

In [6]:
from pyspark.ml.clustering import LDA

In [22]:
from pyspark.sql import SparkSession

spark = SparkSession \
    .builder \
    .appName("Python Spark SQL basic example") \
    .getOrCreate()

In [23]:
spark

In [9]:
type(spark)

pyspark.sql.session.SparkSession

In [24]:
dfs = spark.read.load("NCSC Reports.csv",format='csv', header = "true")

In [25]:
type(dfs)

pyspark.sql.dataframe.DataFrame

In [33]:
dfs.printSchema()

root
 |-- _c0: string (nullable = true)
 |-- Unnamed: 0: string (nullable = true)
 |-- Title: string (nullable = true)
 |-- Article: string (nullable = true)
 |-- topics: string (nullable = true)
 |-- Links: string (nullable = true)



In [10]:
#df2 = spark.read.load("Data/NCSC Reports.csv",
#                     format="csv", sep=";", inferSchema="true", header="true")

In [18]:
#df3 = pickle.load(open('wordlist.p','rb'))

In [20]:
#type(df3)

list

In [35]:
from sparknlp.base import Finisher, DocumentAssembler
from sparknlp.annotator import (Tokenizer, Normalizer,
                                Lemmatizer, LemmatizerModel, StopWordsCleaner)
from pyspark.ml import Pipeline

In [26]:
import nltk
from nltk.corpus import stopwords
from nltk.corpus import words

In [27]:
eng_stopwords = stopwords.words('english')
eng_words = words.words()

In [31]:
documentAssembler = DocumentAssembler() \
     .setInputCol('Article') \
     .setOutputCol('Articles')

tokenizer = Tokenizer() \
     .setInputCols(['Articles']) \
     .setOutputCol('token')

normalizer = Normalizer() \
     .setInputCols(['token']) \
     .setOutputCol('normalized') \
     .setLowercase(True)

stopwords_cleaner = StopWordsCleaner() \
     .setInputCols(['normalized']) \
     .setOutputCol('clean_lemma') \
     .setCaseSensitive(False) \
     .setStopWords(eng_stopwords)

finisher = Finisher() \
     .setInputCols(['clean_lemma']) \
     .setCleanAnnotations(False)

TypeError: 'JavaPackage' object is not callable